Instalão do Playwright para Scrap da página:
<text>"https://www2.unesp.br/portal#!/arex/intercambio/oportunidades/estudantes-de-pos-graduacao/"</text>

In [ ]:
!pip install playwright
!playwright install-deps
!playwright install

# Carrega as oportunidades do arquivo JSON para memória se já existirem

In [ ]:
import json

filename = 'oportunidades.json'
try:
  with open(filename, 'r', encoding='utf-8') as f:
      oportunidades_data = json.load(f)
  print(f"{filename} carregado, total de  {len(oportunidades_data)} oportunidade(s)")

except FileNotFoundError:
  print("O arquivo JSON não foi encontrado.")

# Scrapping

In [ ]:
import asyncio
import json
from playwright.async_api import async_playwright
from urllib.parse import urljoin

In [ ]:
async def scrape_with_playwright():
    oportunidades_data = []
    browser = None
    base_url = "https://www2.unesp.br"

    try:
        # Inicializa o Playwright e abre o navegador
        print("Iniciando o navegador com Playwright...")
        p = await async_playwright().start()
        browser = await p.chromium.launch(headless=True)

        page = await browser.new_page()

        url = "https://www2.unesp.br/portal#!/arex/intercambio/oportunidades/estudantes-de-pos-graduacao/"

        print(f"Navegando para {url}")
        await page.goto(url, wait_until="domcontentloaded")

        # Seletor XPath robusto que busca por <p> ou <div> que contenham
        # um <span> (para a data) e um <a> (para o link)
        XPATH_SELECTOR = '//*[self::p or self::div][span and a[contains(@href, "xajax_exibePagina")]]'

        locator = page.locator(f'xpath={XPATH_SELECTOR}')

        timeout_seconds = 5
        print(f"Aguardando até {timeout_seconds} segundos para os elementos carregarem...")

        try:
            # --- Espera automática do Playwright ---
            await locator.first.wait_for(timeout=timeout_seconds * 1000)

        except Exception as e:
            print(f"Timeout na página ou nenhum elemento encontrado com o seletor.")
            print(f"Erro: {e}")
            if browser:
                await browser.close()
            return

        # --- Se chegamos aqui, os elementos foram encontrados ---
        count = await locator.count()
        print(f"\n--- Encontrados {count} elementos correspondentes ---")

        for i in range(count):
            # Re-localiza o elemento a cada iteração para evitar "stale element"
            current_container = locator.nth(i)

            data_texto_raw = ""
            link_texto = ""

            try:
                # 1. Extrair Data e Título do Link (da página principal)
                data_texto_raw = await current_container.locator("span").first.inner_text()
                data_texto = data_texto_raw.replace("-&nbsp;", "").strip() # Limpa o texto da data

                link_element = current_container.locator("a").first
                link_texto = await link_element.inner_text()

                print(f"\n--- Processando Elemento {i+1}/{count}: {data_texto} | {link_texto.strip()} ---")

                # 2. Clicar no link para abrir o modal
                await link_element.click()

                # 3. Esperar o modal aparecer
                modal_selector = "div.modal-content"
                await page.wait_for_selector(modal_selector, state="visible", timeout=10000)
                modal = page.locator(modal_selector).first

                # 4. Extrair o Título do modal
                title_locator = modal.locator("h1.titulo-pagina").first
                titulo_modal = await title_locator.inner_text()
                print(f"  Título Modal: {titulo_modal.strip()}")

                # 5. Localizar e Extrair o corpo do conteúdo (MODIFICADO)
                body_locator = modal.locator("div.wrapper-conteudo-tinymce").first

                content_parts = []
                # Seleciona todos os filhos diretos do container (ex: <p>, <h2>, <div>, etc.)
                child_elements = body_locator.locator("> *")
                children_count = await child_elements.count()

                for j in range(children_count):
                    element = child_elements.nth(j)
                    text = await element.text_content() # Pega o texto de cada filho
                    if text:
                        cleaned_text = text.strip()
                        if cleaned_text: # Adiciona apenas se não for vazio
                            content_parts.append(cleaned_text)

                # Junta todas as partes com duas quebras de linha
                conteudo = "\n\n".join(content_parts)
                print(f"  Conteúdo (trecho): {conteudo[:100]}...")


                # 6. Extrair todos os links de dentro do corpo
                links_list = []
                # Usa o body_locator (wrapper-conteudo-tinymce) para garantir
                # que pegamos apenas os links do conteúdo
                links_in_body = body_locator.locator("a")
                links_count = await links_in_body.count()

                if links_count > 0:
                    for j in range(links_count):
                        link = links_in_body.nth(j)
                        link_text_modal = await link.inner_text()
                        link_href = await link.get_attribute("href")

                        absolute_href = urljoin(base_url, link_href)

                        links_list.append({
                            "texto": link_text_modal.strip(),
                            "url": absolute_href
                        })
                    print(f"  Links internos: {len(links_list)} encontrados.")
                else:
                    print("  Nenhum link interno encontrado.")

                # 7. Adicionar dados coletados à lista principal
                item_data = {
                    "data": data_texto,
                    "titulo_link": link_texto.strip(),
                    "titulo_modal": titulo_modal.strip(),
                    "conteudo": conteudo, # 'conteudo' agora está formatado
                    "links": links_list
                }
                oportunidades_data.append(item_data)

                # 8. Fechar o modal
                close_button_selector = 'button.btn-close[data-bs-dismiss="modal"]'
                await modal.locator(close_button_selector).click()

                # 9. Esperar o modal desaparecer
                await page.wait_for_selector(modal_selector, state="hidden")
                await asyncio.sleep(0.5) # Uma pequena pausa por segurança

            except Exception as e:
                print(f"  ERRO ao processar o elemento {i+1} ({link_texto.strip()}): {e}")
                print("  Recarregando a página para tentar recuperar...")
                try:
                    await page.reload(wait_until="domcontentloaded")
                    locator = page.locator(f'xpath={XPATH_SELECTOR}') # Re-localiza
                    await locator.first.wait_for(timeout=timeout_seconds * 1000)
                except Exception as reload_e:
                    print(f"  Falha ao recarregar a página. Abortando. {reload_e}")
                    raise

    except Exception as e:
        print(f"Ocorreu um erro geral: {e}")

    finally:
        if browser:
            print("\nFechando o navegador.")
            await browser.close()

    # --- Processamento Final ---
    print("\n--- Scraping Concluído ---")
    if oportunidades_data:
        print(f"Total de {len(oportunidades_data)} oportunidades extraídas.")

        # Converte a lista de dados para JSON
        try:
            json_output = json.dumps(oportunidades_data, indent=2, ensure_ascii=False)

            # Salva o JSON em um arquivo
            output_filename = "oportunidades.json"
            with open(output_filename, "w", encoding="utf-8") as f:
                f.write(json_output)

            print(f"\n✅ Dados salvos com sucesso em: {output_filename}")

        except Exception as json_e:
            print(f"ERRO ao converter dados para JSON: {json_e}")

    else:
        print("Nenhuma oportunidade foi extraída.")

In [ ]:
try:
  # Dentro de notebooks.
  asyncio.get_running_loop()
  await scrape_with_playwright()
except RuntimeError:
  # Fora de notebooks.
  asyncio.run(scrape_with_playwright())

# LLM Local usando Ollama

In [ ]:
!pip install ollama
!pip install colab-xterm

In [ ]:
%load_ext colabxterm
%xterm

<p> comandos para executar no terminal acima </p>

<bold>Instalar lshw:</bold>
sudo apt update
<br>
sudo apt install pciutils lshw

<bold>Verificar se detecta GPU</bold>
lspci -k | grep -EA3 'VGA|3D|Display'
<br>ou<br>
sudo lshw -C display

<bold>carregar o ollama</bold>
curl -fsSL https://ollama.com/install.sh | sh
<br>
ollama pull llama3.2:3b
<br>
ollama serve
<br>
ollama run llama3.2:3b

In [ ]:
import ollama
cliente = ollama.Client()
#model = "llama3.2:3b"
model = "qwen3:8b"

In [ ]:
!ollama pull qwen3:8b

## Define a system message do modelo

In [ ]:
system_message = '''Você é um assistente academico especializado em estudos de pós graduação. Sua tarefa primária é avaliar e classificar oportunidades para estudantes de pós graduação.
    Quando apresentado a uma oportunidade, considere os seguintes critérios de relevancia ao estudante:

    **Foco em pesquisa**: Oferece colaboração em pesquisa, projetos ou acesso a instalações de pesquisa?
    **Desenvolvimento Acadêmico**: Contribui para o desenvolvimento da tese, aprimoramento de habilidades (por exemplo, cursos avançados, workshops) ou oportunidades de publicação?
    **Intercâmbio/Mobilidade Internacional**: Envolve estudos no exterior, conferências internacionais, programas de intercâmbio virtual ou networking global?
    **Financiamento/Bolsas de Estudo**: A instituição oferece bolsas de estudo, auxílios, subsídios ou outro tipo de apoio financeiro para atividades acadêmicas?
    **Desenvolvimento Profissional**: Oferece estágios, treinamentos ou oportunidades de desenvolvimento de carreira relevantes para a área de pós-graduação?

    Sua classificação deve ser clara, concisa e altamente precisa. Concentre-se exclusivamente nas informações fornecidas na descrição da oportunidade.
    Não faça suposições nem infira informações que não estejam explicitamente declaradas.
    Seu trabalho deve abordar diretamente a relevância da oportunidade para um estudante de pós-graduação com base nos critérios acima e categorizá-la, se possível (por exemplo, 'Bolsa de Estudos', 'Intercâmbio de Pesquisa', 'Conferência', 'Curso').

    Cada oportunidade é uma entrada de um arquivo json contendo os seguintes itens:
    {
      "data": str,
      "titulo_link": str,
      "titulo_modal": str,
      "conteudo": str,
      "links": []
    }
    Cada item do array "links" é um objeto contendo os seguintes itens:
    {
      "texto": str,
      "url": str
    }

    Considerando as informações das oportunidades e caso as mesmas intens na lista da chave 'links', utilize a ferramenta fetch_url para agregar informação acessando a página da web expressa na string da chave 'url'.
    Responda EXTRITAMENTE no seguinte formato, **em portugues**:

    Modalidade: <modalidade aferida para a oportunidade>
    Relevancia: <relevancia da oportunidade para o estudante perante as instruções>
    Justificatica: <argumentação breve do porque ela é relavante caso for julgada como tal>
'''

## Cassificação

In [ ]:
classified_opportunities_ollama = []

for i, opportunity in enumerate(oportunidades_data):
    print(f"\n--- Classificando oportunidade número {i+1}/{len(oportunidades_data)} ---")
    print(f"Titulo: {opportunity['titulo_modal']}")

    # Construção do prompt
    comando = "\nDe acordo as suas instruções, classifique a seguinte oportunidade:\n\n"
    full_prompt = f"{system_message}\n{comando}{json.dumps(opportunity)}"

    response = cliente.generate(model=model, prompt=full_prompt)
    temp = {
        "oportunidade": opportunity['titulo_modal'],
        "descricao": opportunity['conteudo'],
        "classificacao": response.response
    }
    classified_opportunities_ollama.append(temp)

print("\nProcesso de classificação concluído.")

In [ ]:
for item in classified_opportunities_ollama:
  print(item['oportunidade'])
  print(item['classificacao'])
  print('\n')

## Salva as oportunidades classificadas em um arquivo JSON

In [ ]:
output_classified_filename = "oportunidades_classificadas_LLM_Ollama_local.json"
with open(output_classified_filename, "w", encoding="utf-8") as f:
    json.dump(classified_opportunities_ollama, f, indent=2, ensure_ascii=False)

print(f"Classificação realizada e salva em: {output_classified_filename}")

# LLM de terceiros usando APIs

In [ ]:
!pip install langchain langgraph langchain_core langchain-openai langchain-google-genai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool
import requests
import json
import time

## Chaves das APIs

Defininir aqui suas chaves de API, código original foi criado no ambiente do google colab, adaptar para o ambiente que o usurário estiver usando.

In [ ]:
from google.colab import userdata
Gkey = userdata.get('GoogleAIKey')
Okey = userdata.get('OpenAIKey')

## Ferramenta de consulta na web

In [ ]:
@tool
def fetch_url(url: str) -> str:
    """Baixa o conteúdo bruto de uma página web."""
    try:
        resp = requests.get(url, timeout=10)
        return resp.text
    except:
        return "ERRO ao acessar URL"

## Escolha do modelo de LLM a se usar

In [ ]:
#llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, api_key=Gkey).bind_tools([fetch_url])
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0, api_key=Gkey)
#llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=Okey)

## Mensagem de sistema detalhando a tarefa

In [ ]:
system_message = SystemMessage(
    '''Você é um assistente academico especializado em estudos de pós graduação. Sua tarefa primária é avaliar e classificar oportunidades para estudantes de pós graduação.
    Quando apresentado a uma oportunidade, considere os seguintes critérios de relevância  ao estudante:

    **Foco em pesquisa**: Oferece colaboração em pesquisa, projetos ou acesso a instalações de pesquisa?
    **Desenvolvimento Acadêmico**: Contribui para o desenvolvimento da tese, aprimoramento de habilidades (por exemplo, cursos avançados, workshops) ou oportunidades de publicação?
    **Intercâmbio/Mobilidade Internacional**: Envolve estudos no exterior, conferências internacionais, programas de intercâmbio virtual ou networking global?
    **Financiamento/Bolsas de Estudo**: A instituição oferece bolsas de estudo, auxílios, subsídios ou outro tipo de apoio financeiro para atividades acadêmicas?
    **Desenvolvimento Profissional**: Oferece estágios, treinamentos ou oportunidades de desenvolvimento de carreira relevantes para a área de pós-graduação?

    Sua classificação deve ser clara, concisa e altamente precisa. Concentre-se exclusivamente nas informações fornecidas na descrição da oportunidade.
    Não faça suposições nem infira informações que não estejam explicitamente declaradas.
    Seu trabalho deve abordar diretamente a relevância  da oportunidade para um estudante de pós-graduação com base nos critérios acima e categorizá-la, se possível, Exemplos:
    Bolsa de Estudos (Financiamento ou Ajuda de Custo), Curso de Idiomas (Teste de Proficiência ou Aprendizado de Línguas), Evento Acadêmico (Conferência, Palestra ou Simpósio), Estágio (Oportunidade de Trabalho ou Desenvolvimento de Carreira), Chamada para Pesquisa (Publicação de Artigos, Prêmio Científico ou Pós-Doutorado), Curso de Curta Duração (Escola de Verão/Inverno ou Workshop)

    Cada oportunidade é uma entrada de um arquivo json contendo os seguintes itens:
    {
      "data": str,
      "titulo_link": str,
      "titulo_modal": str,
      "conteudo": str,
      "links": []
    }
    Cada item do array "links" é um objeto contendo os seguintes itens:
    {
      "texto": str,
      "url": str
    }

    Considerando as informações das oportunidades e caso as mesmas intens na lista da chave 'links', utilize a ferramenta fetch_url para agregar informação acessando a página da web expressa na string da chave 'url'.
    Responda SEMPRE em portugues e EXTRITAMENTE no seguinte formato:

    Modalidade: <modalidade aferida para a oportunidade>
    Relevância : <grau de relevância  da oportunidade para o estudante perante as instruções, cassificada sempre e exclusivamente como IRRELEVANTE, BAIXA, MÉDIA, ALTA ou ESSENCIAL>
    Justificatica: <argumentação breve do porque ela pertence ao grau de relevância  atribuido>

    Caso o usuário especificar alguma modalidade que mereça atenção ou alguma a se ignorar, Exemplo:
    Prioridade do Usuário: <Oportunidades de intercâmbia e Foco em pesquisa>
    Desprezo do Usuário: <Oportunidades de desenvolvimento profissional>

    Neste caso priorizar e ou ignorar as oportunidades que contemple as modalidades mencionadas na hora de atribuir o grau de Relevância .

    Caso algum campo estivar vazio, como por exemplo:
    Prioridade do Usuário: <>
    Desprezo do Usuário: <>

    A escolha do grau de Relevância fica a seu critério.
    '''
)

## Usa a API do modelo escolhido para inferir a classificação

In [ ]:
classified_opportunities_3dparty = []

timeout = 30
for i, opportunity in enumerate(oportunidades_data):
    print(f"\n--- Classificando oportunidade número: {i+1}/{len(oportunidades_data)} ---")
    print(f"Titulo: {opportunity['titulo_modal']}")

    na_prioridades = '''Prioridade do Usuário: <>
    Desprezo do Usuário: <>'''
    prioridades = '''Prioridade do Usuário: <Intercâmbio, Bolsas de estudo>
    Desprezo do Usuário: <Cursos profisionalizantes, Cursos de idiomas, Palestras e Seminários>'''

    messages = [
        system_message,
        HumanMessage(content=[
            {"type": "text", "text": json.dumps(opportunity)+"\n\n"+prioridades}
        ])
    ]
    try:
      response = llm.invoke(messages)
      print(response.content,"\n\n")
      time.sleep(timeout)
      classified_opportunities_3dparty.append(response.content)

    except Exception as e:
      print(f"Erro ao classificar oportunidade: {e}")
      time.sleep(60)

    if i == 15: break

print("\nProcesso de classificação.")

In [ ]:
with open("oportunidades_classificadas_3dparty.json", "w", encoding="utf-8") as f:
    json.dump(classified_opportunities_3dparty, f, indent=2, ensure_ascii=False)

## Tentativas de usar ferramenta de consulta na web (não usada no final pos nao consegui mitigar as inconsistencias e erros de inferencia)

In [ ]:
'''
classified_opportunities = []

for i, opportunity in enumerate(oportunidades_data):
    print(f"\n--- Classificando oportunidade número: {i+1}/{len(oportunidades_data)} ---")
    print(f"Titulo: {opportunity['titulo_modal']}")

    messages = [
        system_message,
        HumanMessage(content=[
            {"type": "text", "text": json.dumps(opportunity)}
        ])
    ]

    classification_result = "N/A"
    try:
        response = llm.invoke(messages)

        while response.tool_calls:
            tool_outputs = []
            for tool_call in response.tool_calls:
                print(f"Chamada da ferramenta: {tool_call.name} com args: {tool_call.args}")
                if tool_call.name == "fetch_url":
                    output = fetch_url.invoke(tool_call.args)
                    tool_outputs.append(ToolMessage(tool_call_id=tool_call.id, content=output))
                else:
                    tool_outputs.append(ToolMessage(tool_call_id=tool_call.id, content=f"Ferramenta {tool_call.name} não suportada"))

            messages.extend(tool_outputs)
            response = llm.invoke(messages)

        if response.content and isinstance(response.content, list) and len(response.content) > 0 and 'text' in response.content[0]:
            try:
                classficada = json.loads(response.content[0]['text'])
                classification_result = classficada
                print(f"Classificação: {json.dumps(classification_result['classificacao'], indent=2, ensure_ascii=False)}\n")
            except json.JSONDecodeError as e:
                print(f"Erro na hora de parsear a resposta JSON '{opportunity['titulo_modal']}': {e}")
                classification_result = {"erro": f"Erro no parser JSON: {e}", "saida_bruta": response.content[0]['text']}
        else:
            print(f"Formatação da resposta da LLM inesperado para a oportunidade: '{opportunity['titulo_modal']}': {response.content}")
            classification_result = {"erro": "Resposta de formatação inesperada", "saida_bruta": response.content}

    except Exception as e:
        print(f"Erro durante a classificação da oportunidade: '{opportunity['titulo_modal']}': {e}")
        classification_result = {"erro": f"Erro geral: {e}"}

    opportunity['classificacao'] = classification_result
    classified_opportunities.append(opportunity)

print("\nProcesso de classificação.")
'''

## Lista os itens classificados

In [ ]:
for item in classified_opportunities_3dparty:
  if 'erro' not in item['classificacao'].keys():
    print(f"Oportunidade: {item['titulo_modal']}")
    print(f"Classificação: {item['classification']['classificacao']['categoria']}")
    print(f"Justificativa: {item['classification']['classificacao']['justificativa']}\n\n")

# Transformers para Zero-Shot

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import pipeline

In [ ]:
criteria_map = {
    "Bolsa de Estudos, Financiamento ou Ajuda de Custo": {
        "desc": "Bolsas e Financiamento",
        "justificativa": "Esta oportunidade oferece suporte financeiro, bolsas integrais/parciais ou auxílio para custos acadêmicos (ex: MEXT, OEA, Brunei)."
    },
    "Curso de Idiomas, Teste de Proficiência ou Aprendizado de Línguas": {
        "desc": "Idiomas e Proficiência",
        "justificativa": "Relacionado ao aprendizado de línguas estrangeiras (PLEU, Instituto Confúcio) ou exames de proficiência (TOEFL, DELE)."
    },
    "Evento Acadêmico, Conferência, Palestra ou Simpósio": {
        "desc": "Eventos e Palestras",
        "justificativa": "Trata-se de um evento pontual, como palestras, feiras estudantis, jornadas acadêmicas ou webinários."
    },
    "Estágio, Oportunidade de Trabalho ou Desenvolvimento de Carreira": {
        "desc": "Estágio e Carreira",
        "justificativa": "Oferta de experiência profissional prática, estágios (ex: Mercosul) ou simulações profissionais (ex: ONU/WIMUN)."
    },
    "Chamada para Pesquisa, Publicação de Artigos, Prêmio Científico ou Pós-Doutorado": {
        "desc": "Pesquisa e Prêmios",
        "justificativa": "Focado em produção científica, submissão de resumos, prêmios acadêmicos (Nature, Mercosul) ou posições de pesquisa avançada."
    },
    "Curso de Curta Duração, Escola de Verão/Inverno ou Workshop": {
        "desc": "Cursos de Curta Duração",
        "justificativa": "Cursos intensivos, escolas de temporada (Summer/Winter Schools) ou workshops específicos que não focam apenas em idiomas."
    }
}

In [ ]:
candidate_labels = list(criteria_map.keys())

In [ ]:
def processar_oportunidades(lista_oportunidades):
    resultados_json = []

    for item in lista_oportunidades:
        texto_completo = f"{item['titulo_modal']}. {item['conteudo']}"

        # Classificação Zero-Shot
        result = classifier(texto_completo, candidate_labels, multi_label=False)

        # Pega o rótulo maior e sua pontuação
        top_label = result['labels'][0]
        score = result['scores'][0]

        # Recuperamos a justificativa baseada no mapa
        info_classificacao = criteria_map[top_label]

        objeto_saida = {
            "oportunidade": f"{item['data'].strip()} {item['titulo_modal']}",
            "descricao": item['conteudo'][:150] + "...", # Truncando
            "classificacao": {
                "categoria": info_classificacao['desc'],
                "confianca_modelo": f"{score*100:.1f}%",
                "justificativa": info_classificacao['justificativa']
            }
        }

        resultados_json.append(objeto_saida)

    return resultados_json

In [ ]:
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

In [ ]:
resultados = processar_oportunidades(oportunidades_data)

In [ ]:
for item in resultados:
  json_str = json.dumps(item, indent=2, ensure_ascii=False)
  print(json_str)

In [ ]:
with open("oportunidades_classificadas_ZeroShot.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, indent=2, ensure_ascii=False)

In [ ]:
filename = 'oportunidades_classificadas_ZeroShot.json'
try:
  with open(filename, 'r', encoding='utf-8') as f:
      oportunidades_zeroshot = json.load(f)
  print(f"{filename} carregado, total de  {len(oportunidades_data)} oportunidade(s)")

except FileNotFoundError:
  print("O arquivo JSON não foi encontrado.")

In [ ]:
def relevanciaZS(oportunidades, labels, t):
  for i, opportunity in enumerate(oportunidades):
    modalidade = opportunity['classificacao']['categoria']
    if modalidade in labels:
      conf = opportunity['classificacao']['confianca_modelo'].replace('%', '')
      if float(conf) >= t:
        print(f"Oportunidade: {opportunity['oportunidade']} é relevante!")
        print(f"Modalidade: {modalidade}")
        print(f"Confiança do modelo: {conf}%")
        print(f"Justificativa: {opportunity['classificacao']['justificativa']}\n")

In [ ]:
# categorias disponiveis são:
#
# Bolsas e Financiamento
# Idiomas e Proficiência
# Eventos e Palestras
# Estágio e Carreira
# Pesquisa e Prêmios
# Cursos de Curta Duração

relevanciaZS(oportunidades_zeroshot, ['Bolsas e Financiamento', 'Estágio e Carreira'], 30)